In [3]:
import numpy as np
import pandas as pd
import re
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn.functional as F
from tqdm import tqdm
from torchtext.vocab import GloVe
import matplotlib.pyplot as plt
from spellchecker import SpellChecker
import warnings
from transformers import AutoTokenizer, AutoModelForSequenceClassification

warnings.filterwarnings("ignore")

c:\Users\abhij\AppData\Local\Programs\Python\Python39\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\abhij\AppData\Local\Programs\Python\Python39\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [4]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def remove_misspelled_words(text):
    words = text.split()
    corrected_words = [word for word in words if word.isdigit() or word.lower() in spell]
    return ' '.join(corrected_words)


def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(predictions, dim=1).item()
    return sentiment

In [6]:
file_path = 'D:\\BSES - Data Analyst\\Sentiment Analysis\\Data\\Twitter_Comments.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')
df['Customer_Text'] = df['Customer_Text'].astype(str)
df

,Customer_Text,Department,Complaint_Type,BSES_Text,Sentiment
0,i am very shocked due to my electricity bill.....,Business,"Bill Related (Bill Download Issue, Wrong Bill)",sorry for the inconvenience caused. please pro...,Negative
1,can we know about the new base/guidelines abou...,Business,"Bill Related (Bill Download Issue, Wrong Bill)",it would be great if you could share your ca a...,Negative
2,\nit is third day of low voltage fluctuation i...,O&M,Voltage Fluctuation (Low & High),\nwe're getting this checked and will get back...,Negative
3,à¤à¤• à¤¦à¤® à¤®à¤œà¤¾à¤• à¤¬à¤¨à¤¾ à¤¦à¤¿à¤...,O&M,Power Outage,hey your complaint has been already registered...,Negative
4,à¤à¤• à¤¦à¤® à¤®à¤œà¤¾à¤• à¤¬à¤¨à¤¾ à¤¦à¤¿à¤...,O&M,Power Outage,we'll surely look into this for you. be assure...,NaN
...,...,...,...,...,...
50148,"here,s no electrical supply qhy you cut the el...",O&M,Power Outage,sorry for the inconvenience caused. please pro...,Negative
50149,no power for ca no 151791523,O&M,Power Outage,it would be great if you could share your cont...,Negative
50150,no electricity in our area my ca no is 101424877,O&M,Power Outage,sorry for the inconvenience caused. we have no...,Negative
50151,still no update\nwhen would be power restored,O&M,Power Outage,sorry for the inconvenience caused. please pro...,Negative


In [21]:
df.dropna(inplace=True)
df.drop(["Department", "Complaint_Type ", "BSES_Text", "Sentiment"], axis='columns', inplace=True)
df

,Customer_Text
0,i am very shocked due to my electricity bill.....
1,can we know about the new base/guidelines abou...
2,\nit is third day of low voltage fluctuation i...
3,à¤à¤• à¤¦à¤® à¤®à¤œà¤¾à¤• à¤¬à¤¨à¤¾ à¤¦à¤¿à¤...
5,"\nnow that 30/6/20 is over, lab test must have..."
...,...
50148,"here,s no electrical supply qhy you cut the el..."
50149,no power for ca no 151791523
50150,no electricity in our area my ca no is 101424877
50151,still no update\nwhen would be power restored


In [22]:
df['Customer_Text'] = df['Customer_Text'].apply(clean_text)
df = df[df['Customer_Text'].str.strip() != '']
df = df[df['Customer_Text'].str.split().apply(len) > 1]
df

,Customer_Text
0,i am very shocked due to my electricity bill l...
1,can we know about the new baseguidelines about...
2,it is third day of low voltage fluctuation in ...
5,now that 30620 is over lab test must have be d...
7,no power supply in our area since last 30 min ...
...,...
50148,heres no electrical supply qhy you cut the ele...
50149,no power for ca no 151791523
50150,no electricity in our area my ca no is 101424877
50151,still no update when would be power restored


In [23]:
spell = SpellChecker()
df['Customer_Text'] = df['Customer_Text'].apply(remove_misspelled_words)
df

,Customer_Text
0,i am very shocked due to my electricity bill l...
1,can we know about the new about electricity bi...
2,it is third day of low voltage fluctuation in ...
5,now that 30620 is over lab test must have be d...
7,no power supply in our area since last 30 no 1...
...,...
50148,heres no electrical supply you cut the electri...
50149,no power for no 151791523
50150,no electricity in our area my no is 101424877
50151,still no update when would be power restored


In [14]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
max_len = max(len(seq) for seq in df['Customer_Text'])
print(max_len)

5337


In [26]:
sentiments = []
for text in tqdm(df['Customer_Text'], desc="Processing sentiments"):
    sentiment = get_sentiment(text)
    sentiments.append(sentiment)

df['Sentiment'] = sentiments
df

Processing sentiments: 100%|██████████| 34609/34609 [41:08<00:00, 14.02it/s]  


,Customer_Text,Sentiment
0,i am very shocked due to my electricity bill l...,0
1,can we know about the new about electricity bi...,0
2,it is third day of low voltage fluctuation in ...,0
5,now that 30620 is over lab test must have be d...,1
7,no power supply in our area since last 30 no 1...,0
...,...,...
50148,heres no electrical supply you cut the electri...,0
50149,no power for no 151791523,0
50150,no electricity in our area my no is 101424877,0
50151,still no update when would be power restored,0


In [27]:
sentiment_labels = {0: 'negative', 1: 'neutral', 2: 'positive'}
df['Sentiment_Label'] = df['Sentiment'].map(sentiment_labels)

In [28]:
df

,Customer_Text,Sentiment,Sentiment_Label
0,i am very shocked due to my electricity bill l...,0,negative
1,can we know about the new about electricity bi...,0,negative
2,it is third day of low voltage fluctuation in ...,0,negative
5,now that 30620 is over lab test must have be d...,1,neutral
7,no power supply in our area since last 30 no 1...,0,negative
...,...,...,...
50148,heres no electrical supply you cut the electri...,0,negative
50149,no power for no 151791523,0,negative
50150,no electricity in our area my no is 101424877,0,negative
50151,still no update when would be power restored,0,negative


In [30]:
file_path = 'D:\\BSES - Data Analyst\\Sentiment Analysis\\Data\\Twitter_Comments.xlsx'
old_data = pd.read_excel(file_path, engine='openpyxl')

In [33]:
old_data['Customer_Text'] = old_data['Customer_Text'].astype(str)
old_data.dropna(inplace=True)
old_data.drop(["Department", "Complaint_Type ", "BSES_Text", "Sentiment"], axis='columns', inplace=True)
old_data

,Customer_Text
0,i am very shocked due to my electricity bill.....
1,can we know about the new base/guidelines abou...
2,\nit is third day of low voltage fluctuation i...
3,à¤à¤• à¤¦à¤® à¤®à¤œà¤¾à¤• à¤¬à¤¨à¤¾ à¤¦à¤¿à¤...
5,"\nnow that 30/6/20 is over, lab test must have..."
...,...
50148,"here,s no electrical supply qhy you cut the el..."
50149,no power for ca no 151791523
50150,no electricity in our area my ca no is 101424877
50151,still no update\nwhen would be power restored


In [34]:
old_data['Customer_Text'] = old_data['Customer_Text'].apply(clean_text)
old_data = old_data[old_data['Customer_Text'].str.strip() != '']
old_data = old_data[old_data['Customer_Text'].str.split().apply(len) > 1]
old_data

,Customer_Text
0,i am very shocked due to my electricity bill l...
1,can we know about the new baseguidelines about...
2,it is third day of low voltage fluctuation in ...
5,now that 30620 is over lab test must have be d...
7,no power supply in our area since last 30 min ...
...,...
50148,heres no electrical supply qhy you cut the ele...
50149,no power for ca no 151791523
50150,no electricity in our area my ca no is 101424877
50151,still no update when would be power restored


In [43]:
newData = pd.DataFrame({
    'Customer_Text': old_data['Customer_Text'],
    'Sentiment': df["Sentiment_Label"]
})

In [46]:
newData.to_csv('D:\\BSES - Data Analyst\\Sentiment Analysis\\Data\\Roberta Data.csv', index=False)